In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\DataScience\\Projects\\folder\\AIDRP-AI_Driven_Diabetes_Readmission_Prevention'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from src.AIDRP.constants import *
from src.AIDRP.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config
        

In [6]:
import os
import urllib.request as request
import zipfile
from src.AIDRP.logging import logger
from src.AIDRP.utils.common import get_size

class DataIngestion :
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_config = DataIngestion(data_ingestion_config)
    data_ingestion_config.download_file()
    data_ingestion_config.extract_zip_file()
except Exception as e:
    raise e

[2024-02-12 20:42:34,507 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-12 20:42:34,514 : INFO : common : yaml file: params.yaml loaded successfully]


[2024-02-12 20:42:34,525 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-12 20:42:34,539 : INFO : common : created directory at: artifacts]
[2024-02-12 20:42:34,540 : INFO : common : created directory at: artifacts/data_ingestion]
[2024-02-12 20:42:39,158 : INFO : 2870775349 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3169121
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a40b2cf0e6a5c1d725cbcad306a9fa130f012db4fe08de0e93a1859d62269bdd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DA96:2E0CD:108DB:168DD:65CA3564
Accept-Ranges: bytes
Date: Mon, 12 Feb 2024 15:12:37 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4728-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1707750757.875904,VS0,VE369
Vary: Authorization,Accep